In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import bernoulli, binom
import math


In [11]:
df1 = pd.read_csv('data_policy1.csv', header = None).rename(columns = {0: 'st', 1: 'at', 2: 'rt', 3: 'st1'})
df2 = pd.read_csv('data_policy2.csv', header = None).rename(columns = {0: 'st', 1: 'at', 2: 'rt', 3: 'st1'})

In [78]:
def modelbased(data, alpha = 0, T = 10**6):
    gamma = 0.95
    S = 5
    A = 2
    data = data[0:T]

    N_ss = np.ones((A, S, S))
    for a in range(A):
        for s in range(S):
            for s_m in range(S):
                N_ss[a][s][s_m] += len(data[(data['at'] == a) & (data['st'] == s) & (data['st1'] == s_m)])
    P_ss = np.array([[[((N_ss[a][s][s_m] + alpha) / (sum(N_ss[a][s]) + (alpha * S))) for s_m in range(S)] for s in range(S)] for a in range(A)])

    r_s = np.ones((A, S))
    for a in range(A):
        for s in range(S):
            r_s[a][s] = (sum(data[(data['st'] == s) & (data['at'] == a)]['rt']) + alpha) / (sum(N_ss[a][s]) + alpha)

    Q_t = np.ones((A, S))
    for a in range(A):
        Q_t[a] = np.linalg.inv(np.identity(S) - gamma * P_ss[a]) @ np.array(r_s[a])

    return Q_t

def QL(data, alpha0):
    gamma = 0.95
    S = 5
    A = 2
    Q_t_arr = np.zeros((A, S))
    Q_t = np.zeros((A, S))
    for t in range(len(data)):
        s = data['st'][t]
        if alpha0 == "alpha_a":
            alpha = 10 / ((t+1)**(2/3) + 1)
        if alpha0 == "alpha_b":
            N_t = sum((data[:t+1]['st'] == s) & (data[:t+1]['at'] == data['at'][t]))
            alpha = 10 / (N_t**(2/3) + 1)
            
        #get V_{t+1}
        a = data['at'][t]
        Q_t[a][s] += alpha * (data['rt'][t] + (gamma * (Q_t[a][data['st1'][t]])) - Q_t[a][s])
        Q_t_arr = np.vstack([Q_t_arr, Q_t])

    return Q_t

In [10]:
#P_ss[a][s][s']
P_ss = np.array([[[0.95, 0.05, 0, 0], 
                  [0.95, 0, 0.05, 0], 
                  [0, 0.95, 0, 0.05], 
                  [0, 0, 0.95, 0.05]],
                 [[0.6, 0.4, 0, 0],
                  [0.05, 0.55, 0.4, 0],
                  [0, 0.05, 0.55, 0.4],
                  [0, 0, 0.4, 0.6]]])

#r_ss[a][s]
r_s = np.array([[1, 0, 0, 0], 
               [0.05, 0, 0, 0]])

In [80]:
data = df2
print(modelbased(data))
print(QL(data, alpha0 = "alpha_b"))


[[0.99818507 0.94812179 0.90053751 0.85553937 0.81445783]
 [5.36185894 6.06241334 6.91982965 7.98749679 9.19382841]]
[[ 1.          0.95        0.9025      0.857375    0.81450625]
 [ 6.06266859  6.80153954  7.85042179  8.73647243 10.21527246]]


In [16]:
np.zeros((2,4,4))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [48]:
[1 if True else 0 for i in range(10)]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]